In [160]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [164]:
# Step3 HTMLから株価データを抽出する関数の作成
def extract_stock_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    elems = soup.select('div[class="highcharts-tooltip"] > span > table > tbody > tr:nth-child(1) > td')
    # 日付取得
    date= elems[0].text
    dateelem = date.split('/')
    date_f = f'{dateelem[0]}年{dateelem[1].zfill(2)}月{dateelem[2].zfill(2)}日'
    #値取得
    hajime =float(elems[1].text.split(':')[1].strip().replace(',',''))
    takane =float(elems[2].text.split(':')[1].strip().replace(',',''))
    yasune =float(elems[3].text.split(':')[1].strip().replace(',',''))
    owarine =float(elems[4].text.split(':')[1].strip().replace(',',''))
    return date_f,hajime,takane,yasune,owarine

# print(extract_stock_data(chrome_driver.page_source))


In [167]:
def get_stock_values(output):
    # Chrome立ち上げ
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_driver = webdriver.Chrome(options=chrome_options)
    # 日経HPアクセス
    chrome_driver.get('https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month')
    wait=WebDriverWait(chrome_driver,30)
    wait.until(
    EC.visibility_of_element_located(
        (By.XPATH, '/html/body/div[7]/div/div/div/div[3]/div/div[3]/div[1]/h2')
    )
    )
    # 右端にマウスを移動
    from selenium.webdriver.common.action_chains import ActionChains
    graph = chrome_driver.find_elements(By.CSS_SELECTOR,"g[class=highcharts-grid]")[1]
    actions = ActionChains(chrome_driver)
    actions.move_to_element(graph)
    actions.move_by_offset(+(graph.rect['width'] // 2),0)
    actions.perform()
    graph_width = graph.rect['width']
    #値取得
    for i in range(int(graph_width/5)):
        info = extract_stock_data(chrome_driver.page_source)
        output.total.append(info)
        if not info:
            continue
        # print(i, info)
        # マウスを左に移動させ繰り返し
        actions = ActionChains(chrome_driver)
        actions.move_by_offset(-5,0)
        actions.perform()

In [168]:
# main 
import time,datetime
start = time.time() 
class output:
    total=[]
output_instance = output()
  
get_stock_values(output_instance)
t = int(time.time() - start)
print(f'スクレイピングにかかった時間={datetime.timedelta(seconds=t)}')
import pprint
pprint.pprint(output_instance.total)

スクレイピングにかかった時間=0:00:43
[('2023年07月07日', 32450.64, 32730.25, 32327.9, 32388.42),
 ('2023年07月06日', 33058.4, 33079.45, 32637.72, 32773.02),
 ('2023年07月05日', 33165.06, 33389.22, 33041.14, 33338.7),
 ('2023年07月04日', 33512.26, 33576.45, 33338.78, 33422.52),
 ('2023年07月03日', 33517.6, 33762.81, 33510.55, 33753.33),
 ('2023年06月30日', 33068.36, 33232.89, 32918.77, 33189.04),
 ('2023年06月29日', 33306.84, 33527.98, 33185.2, 33234.14),
 ('2023年06月28日', 32807.84, 33193.99, 32642.87, 33193.99),
 ('2023年06月27日', 32629.96, 32689.1, 32306.99, 32538.33),
 ('2023年06月26日', 32647.08, 32884.73, 32392.72, 32698.81),
 ('2023年06月23日', 33458.35, 33533.47, 32575.56, 32781.54),
 ('2023年06月22日', 33438.01, 33641.46, 33232.19, 33264.88),
 ('2023年06月21日', 33200.63, 33657.87, 33154.68, 33575.14),
 ('2023年06月21日', 33200.63, 33657.87, 33154.68, 33575.14),
 ('2023年06月20日', 33269.57, 33474.74, 33089.02, 33388.91),
 ('2023年06月19日', 33768.69, 33772.89, 33231.56, 33370.42),
 ('2023年06月16日', 33399.15, 33772.76, 33186.93, 33706.08